In [1]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [2]:
import glob
import pandas as pd

In [3]:
sorted(glob.glob("data/inputs/*_response.csv"))

['data/inputs/100k_response - 100k_response.csv',
 'data/inputs/200k_response - 200k_response.csv',
 'data/inputs/300k_response - 300k_response.csv',
 'data/inputs/400k_response - 400k_response.csv',
 'data/inputs/500k_response - 500k_response.csv']

In [ ]:
df_main = pd.DataFrame()

for input_path in sorted(glob.glob("data/inputs/*_response.csv")):
    df = pd.read_csv(input_path)
    df_main = pd.concat([df_main, df], axis=0, ignore_index=True)
    print(df.shape, df_main.shape)

(100000, 8) (100000, 8)
(100000, 8) (200000, 8)
(100000, 8) (300000, 8)
(100000, 8) (400000, 8)
(100000, 8) (500000, 8)


In [55]:
df_main.head()

,source_url,source_title,response_code,backlinks_num,domains_num,last_seen,external_num,internal_num
0,https://dice.fm/,Discover live events and gigs | Buy tickets on...,200,20866,2857,1734254195,6,27
1,http://dice.fm/,NaN,301,15771,820,1731420176,0,0
2,https://dice.fm/?lng=en,Discover live events and gigs | Buy tickets on...,200,1598,242,1734189665,6,27
3,https://dice.fm/?lng=it,Discover live events and gigs | Buy tickets on...,200,400,159,1733961593,6,29
4,https://dice.fm/event/moxbl-boiler-room-miami-...,BOILER ROOM MIAMI: III POINTS Tickets | From F...,200,418,94,1732237042,0,0


In [56]:
df_main.shape

(500000, 8)

In [57]:
df_filtered = df_main[df_main['source_url'].str.contains(r'^https://dice\.fm/event/.*$', regex=True)]
print(df_filtered.shape)
df_filtered = df_filtered[["source_url"]]
df_filtered['source_url'] = df_filtered['source_url'].str.replace(r'\?.*$', '', regex=True)
print(df_filtered.shape)

(315945, 8)
(315945, 1)


In [58]:
df_filtered.sample(10)

,source_url
378887,https://dice.fm/event/bbbram-donaeo-friends-ca...
399636,https://dice.fm/event/deqbq-havent-heard-of-it...
420538,https://dice.fm/event/eogno6-islington-comedy-...
386977,https://dice.fm/event/boqxm
263715,https://dice.fm/event/5337qk-tagua-tagua-heave...
256874,https://dice.fm/event/3oa9p-youth-27th-mar-the...
53848,https://dice.fm/event/33v7x
302660,https://dice.fm/event/75am7-over-easy-20th-apr...
256981,https://dice.fm/event/3ok56p-fly-sam-alfred-to...
299456,https://dice.fm/event/6vrkb-enrico-capuano-tam...


In [59]:
df_filtered.duplicated().shape

(315945,)

In [37]:
df_filtered[df_filtered["source_url"]=="https://dice.fm/event/7gwep-tyler-ramsey-28th-mar-gold-diggers-los-angeles-tickets"]

,source_url
310745,https://dice.fm/event/7gwep-tyler-ramsey-28th-...


# Merging

In [17]:
df_output = pd.read_excel("data/outputs/dice_events_DICE Event Links 1st batch - Sheet1_20241217_171759.xlsx")
df_input = pd.read_csv("data/inputs/Dice Batch_2 - Sheet1.csv")

In [70]:
df_output.head()

,id,url,processed,processing,sheet_name,event_name,date,location,organiser_name,_rid,_self,_etag,_attachments,_ts,error
0,2151e4dbe8827f729394280a4bc1154c22fbdd768f3f53...,https://dice.fm/event/pnxwy-genova-summer-live...,True,0.0,DICE Event Links 1st batch - Sheet1,Genova Summer Live 2024,"Sat, Jul 6, 2:50 PM","Calata Gadda, 16128 Genova GE, Italy",Tutte le età,66UjANGx7eIP6gAAAAAAAA==,dbs/66UjAA==/colls/66UjANGx7eI=/docs/66UjANGx7...,"""16004861-0000-0a00-0000-675d74d80000""",attachments/,1734178008,NaN
1,c0e422821c3b15ac37cb7122e0c2711586f9bc5b8a55b4...,https://dice.fm/event/lp96x-lima-limo-presents...,True,0.0,DICE Event Links 1st batch - Sheet1,Lima Limo Presents: Loa & The Vernon Spring,"Tue, Jun 25, 7:30 PM","10A Bradbury Street, London N16 8JN",Presented by Lima Limo,66UjANGx7eIQ6gAAAAAAAA==,dbs/66UjAA==/colls/66UjANGx7eI=/docs/66UjANGx7...,"""16004961-0000-0a00-0000-675d74d90000""",attachments/,1734178009,NaN
2,f5a6d31da667dc5e5dd27e309830e29fff7f5d20fa9ba5...,https://dice.fm/event/bnl3v-rock-arena-feat-di...,True,0.0,DICE Event Links 1st batch - Sheet1,ROCK ARENA FEAT DIAFRAMMA,"Sat, Jan 7, 2023, 9:00 PM","Viale J. F. Kennedy, 52, 00043 Ciampino RM, Italy",Presentato da ORION LIVE S.R.L..,66UjANGx7eIR6gAAAAAAAA==,dbs/66UjAA==/colls/66UjANGx7eI=/docs/66UjANGx7...,"""16004761-0000-0a00-0000-675d74d80000""",attachments/,1734178008,NaN
3,12c236bbb0ff28967dc66b155e443938bceb4b89f76410...,https://dice.fm/event/qdk89-firmament-27th-mar...,True,0.0,DICE Event Links 1st batch - Sheet1,Denial presents: DAMEFRISØR,"Wed, Mar 27, 7:30 PM","96-98 Pentonville Rd, London N1 9JB",Presented by Denial,66UjANGx7eIS6gAAAAAAAA==,dbs/66UjAA==/colls/66UjANGx7eI=/docs/66UjANGx7...,"""16004a61-0000-0a00-0000-675d74d90000""",attachments/,1734178009,NaN
4,cc6fcd9e900bbcfebe449d0fedf1bde046822e26f6b135...,https://dice.fm/event/gd3bb-comedy-special-rec...,True,0.0,DICE Event Links 1st batch - Sheet1,Comedy Special Recordings - Full Day Ticket,"Sun, Jul 16, 2023, 2:00 PM","Old Trades Hall, Valette Street, London E9 6NU",Presented by LNZRT LTD.,66UjANGx7eIT6gAAAAAAAA==,dbs/66UjAA==/colls/66UjANGx7eI=/docs/66UjANGx7...,"""1600af61-0000-0a00-0000-675d74e10000""",attachments/,1734178017,NaN


In [71]:
df_input.head()

,Links
0,https://dice.fm/event/dj-yoda-presents-the-str...
1,https://dice.fm/event/dj-yoda-the-heatwave-27t...
2,https://dice.fm/event/dj-yodas-halloween-night...
3,https://dice.fm/event/dj-zinc-cause-affect-5th...
4,https://dice.fm/event/dj-zinc-trust-me-i-was-t...


In [72]:
common_urls = set(df_output['url']).intersection(set(df_input['Links']))

In [73]:
len(common_urls), df_input.shape

(50533, (188986, 1))

In [74]:
df1_filtered = df_input[~df_input['Links'].isin(common_urls)]

In [77]:
df1_filtered.head()

,Links
8,https://dice.fm/event/dk23q-dave-9th-may-the-e...
10,https://dice.fm/event/dk27g-mallory-knox-17th-...
11,https://dice.fm/event/dk28q-stella-donnelly-7t...
17,https://dice.fm/event/dk2x6-born-ruffians-8th-...
18,https://dice.fm/event/dk2x6-born-ruffians-8th-...


In [78]:
df1_filtered.to_csv("data/inputs/Dice Batch 3.csv")

# Last Batch

In [8]:
df_output = pd.read_csv("data/inputs/Dice Batch 3.csv")
df_output.head()

,Unnamed: 0,Links
0,8,https://dice.fm/event/dk23q-dave-9th-may-the-e...
1,10,https://dice.fm/event/dk27g-mallory-knox-17th-...
2,11,https://dice.fm/event/dk28q-stella-donnelly-7t...
3,17,https://dice.fm/event/dk2x6-born-ruffians-8th-...
4,18,https://dice.fm/event/dk2x6-born-ruffians-8th-...


In [13]:
df_input = pd.read_csv('data/inputs/100k_response - 100k_response.csv')
df_input = df_input[df_input['source_url'].str.contains(r'^https://dice\.fm/event/.*$', regex=True)]
print(df_input.shape)

df_input = df_input[["source_url"]]
df_input['source_url'] = df_input['source_url'].str.replace(r'\?.*$', '', regex=True)
print(df_input.shape)

df_input.head()

(46333, 8)
(46333, 1)


,source_url
4,https://dice.fm/event/moxbl-boiler-room-miami-...
5,https://dice.fm/event/25gxp-spring-festival-lu...
7,https://dice.fm/event/79836-radio-company-live...
9,https://dice.fm/event/rr62r-retro5pective-20-y...
10,https://dice.fm/event/6ap53-bright-magus-in-co...


In [14]:
common_urls = set(df_output['Links']).intersection(set(df_input['source_url']))

In [16]:
len(common_urls), df_input.shape, df_output.shape

(2, (46333, 1), (138453, 2))